In [1]:
!pip install langchain_community
!pip install langchain faiss-cpu sentence-transformers
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.9/471.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [2]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:
import os
import time
from typing import List, Tuple, Any, Mapping, Optional
import sys

In [4]:
import json
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import LLMResult, Generation

In [5]:
DOCUMENTS_DIR = "documents"
os.makedirs(DOCUMENTS_DIR, exist_ok=True)

print("done")

done


In [6]:
class MockLLM(BaseLLM):

    language: str = "en"

    @property
    def _llm_type(self) -> str:
        return "mock_llm"


    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        **kwargs: Any,
    ) -> LLMResult:
        prompt = prompts[0]

        if "stock" in prompt.lower() or "أسهم" in prompt:

            if self.language == "ar":
                mock_text = "عذراً، لا يمكنني الإجابة على هذا السؤال لأنه خارج نطاق وثائقنا. يرجى الاستعلام عن المنتجات أو الضمانات المذكورة."
            else:
                mock_text = "I apologize, but this question is outside the scope of the provided documents. Please try a question related to our products or warranties."


        elif self.language == "ar":

            mock_text = "الإجابة الوهمية: الضمان المقدم على المكونات الإلكترونية هو خمس سنوات. نعم، الشركة حاصلة على شهادة ترشيد. (إجابة وهمية بناءً على المحتوى المسترجع: doc_ar_guarantee.txt)."


        else:
            mock_text = "Mock Answer: The primary product mentioned is LED Streetlight Poles, and the warranty is five years for electronic components. (Mock response based on retrieved content)."

        return LLMResult(generations=[[Generation(text=mock_text)]])

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"language": self.language}

print("The MockLLM class, committed to BaseLLM, and corrected rejection logic were defined.")

The MockLLM class, committed to BaseLLM, and corrected rejection logic were defined.


In [7]:
#arabic Decument
doc_ar_content = """
الشركة: الأنوار للتقنيات الضوئية
المنتج الرئيسي: مصابيح الشوارع بتقنية LED.
الضمان: خمس سنوات على المكونات الإلكترونية.
الشهادات: حاصلون على شهادة SASO وشهادة ترشيد.
"""
with open(os.path.join(DOCUMENTS_DIR, "doc_ar_guarantee.txt"), "w", encoding="utf-8") as f:
    f.write(doc_ar_content)

#English Decument
doc_en_content = """
Company: Alrouf Lighting Technology Pvt Ltd.
Primary Product: LED Streetlight Poles, Model ALR-SL-90W.
Location: Our main office is located in Jeddah, Saudi Arabia.
Delivery: Estimated delivery time for Dammam projects is 4 weeks.
"""
with open(os.path.join(DOCUMENTS_DIR, "doc_en_product.txt"), "w", encoding="utf-8") as f:
    f.write(doc_en_content)

#Document 3: Mix (AR/EN) of requests
doc_mix_content = """
For quotation requests (RFQ), please contact omar@client.com.
الرجاء طلب عرض سعر (Quotation) عبر إرسال بريد إلكتروني إلى contact@alrouf.com.
"""
with open(os.path.join(DOCUMENTS_DIR, "doc_mix_contact.txt"), "w", encoding="utf-8") as f:
    f.write(doc_mix_content)

print(f"Three sample documents were created in the folder.'{DOCUMENTS_DIR}'.")

Three sample documents were created in the folder.'documents'.


In [8]:
def build_knowledge_base() -> FAISS:

    print("Start building the knowledge base")
    all_documents = []

    # Decoument(Loader)
    for filename in os.listdir(DOCUMENTS_DIR):
        file_path = os.path.join(DOCUMENTS_DIR, filename)
        if filename.endswith(".txt"):
            loader = TextLoader(file_path, encoding='utf-8')
        elif filename.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        else:
            continue

        loaded_docs = loader.load()
        for doc in loaded_docs:
            doc.metadata["source"] = filename
        all_documents.extend(loaded_docs)

    print(f"Download done {len(all_documents)}")

    #Division of documents(Chunk)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_documents(all_documents)
    print(f"It was divided into{len(chunks)} Chunks.")

    #  Embedding service: Using a local model (All-MiniLM-L6-v2)
    print("Loading the local embed template and index building...")
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    # Creating an index using FAISS
    vector_store = FAISS.from_documents(chunks, embeddings)

    print("Building the FAISS knowledge base has been successfully completed.")
    return vector_store


# 2. Manual RAG Execution

def query_knowledge_base(vector_store: FAISS, question: str, language: str) -> Tuple[str, List[str], float]:
    """
    ينفذ خط أنابيب RAG يدوياً: Retrieve -> Format Prompt -> Generate.
    """
    start_time = time.time()

    # 1. (Retrieve)
    retrieved_docs = vector_store.similarity_search(question, k=3)

    # (Context)
    context_text = "\n---\n".join([doc.page_content for doc in retrieved_docs])

    # (Format Prompt)
    lang_hint = "العربية" if language.lower() == 'ar' else "English"

    prompt_template = """
    أنت نظام أسئلة وأجوبة دقيق ولبق لشركة إضاءة.
    استخدم قطع السياق التالية للإجابة على السؤال في نهاية المطاف.

    1. الإجابة يجب أن تكون باللغة المطلوبة: {language_hint}.
    2. يجب عليك **اقتباس المصادر** التي استخدمتها للإجابة (اسم الملف).
    3. إذا كان السؤال لا يتعلق بمحتوى الوثائق المقدمة، فارفض الإجابة بلباقة باستخدام العبارة "خارج النطاق".

    السياق:
    {context}

    السؤال: {question}
    الإجابة ({language_hint}):
    """

    final_prompt = prompt_template.format(
        language_hint=lang_hint,
        context=context_text,
        question=question
    )

    # (Generate)
    llm = MockLLM(language=language)
    answer = llm.invoke(final_prompt)

    end_time = time.time()
    latency = end_time - start_time

    # Extracting sources
    sources = list(set([doc.metadata.get("source", "Unknown Source") for doc in retrieved_docs]))

    return answer, sources, latency

print("All RAG pipeline functions have been manually defined.")

All RAG pipeline functions have been manually defined.


In [9]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# for LLM
from langchain_core.language_models.llms import BaseLLM
from langchain_core.outputs import LLMResult, Generation
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

In [10]:
# Building a knowledge base (Ingest, Chunk, Embed, Index)
vector_store = build_knowledge_base()
#1. Scenario A: Arabic question (within scope)
LANGUAGE_AR = "ar"
QUESTION_AR = "ما هو الضمان المقدم لمنتجاتكم وهل لديكم شهادة ترشيد؟"

answer_ar, sources_ar, latency_ar = query_knowledge_base(vector_store, QUESTION_AR, LANGUAGE_AR)

print("\n" + "=".center(60, "="))
print(" Arabic question:".center(60))
print("=".center(60, "="))
print(f" Query: {QUESTION_AR}")
print(f" Answer:\n{answer_ar}")
print(f"\n Sources (Citations): {sources_ar}")
print(f"- Performance Report - Response Time: {latency_ar:.2f} Second\n Cost: $0.00 (where Mock LLM and Embeddings are local)")


# 2. Scenario B: English question (outside scope)

LANGUAGE_EN = "en"
QUESTION_EN = "What are the latest stock market trends for tech companies?"

answer_en, sources_en, latency_en = query_knowledge_base(vector_store, QUESTION_EN, LANGUAGE_EN)

print("\n" + "=".center(60, "="))
print("Scenario B: English question (out of scope)".center(60))
print("=".center(60, "="))
print(f" Query: {QUESTION_EN}")
print(f" Answer:\n{answer_en}")
print(f"\n Sources (Citations): {sources_en}")
print(f"Performance Report - Response Time {latency_en:.2f} Second\n Cost: $0.00 (where Mock LLM and Embeddings are local)")

Start building the knowledge base
Download done 3
It was divided into3 Chunks.
Loading the local embed template and index building...


/tmp/ipython-input-231235743.py:34: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Building the FAISS knowledge base has been successfully completed.

                      Arabic question:                      
 Query: ما هو الضمان المقدم لمنتجاتكم وهل لديكم شهادة ترشيد؟
 Answer:
الإجابة الوهمية: الضمان المقدم على المكونات الإلكترونية هو خمس سنوات. نعم، الشركة حاصلة على شهادة ترشيد. (إجابة وهمية بناءً على المحتوى المسترجع: doc_ar_guarantee.txt).

 Sources (Citations): ['doc_ar_guarantee.txt', 'doc_mix_contact.txt', 'doc_en_product.txt']
- Performance Report - Response Time: 0.07 Second
 Cost: $0.00 (where Mock LLM and Embeddings are local)

        Scenario B: English question (out of scope)         
 Query: What are the latest stock market trends for tech companies?
 Answer:
I apologize, but this question is outside the scope of the provided documents. Please try a question related to our products or warranties.

 Sources (Citations): ['doc_ar_guarantee.txt', 'doc_mix_contact.txt', 'doc_en_product.txt']
Performance Report - Response Time 0.04 Second
 Cost: $0.00 (w